In [4]:
from utilities.app_state import RouterRespons
from utilities.LLMProvider import LLMProvider
from chains import  get_router_chain

ValidationError: 6 validation errors for Settings
TOGETHER_API_KEY
  Field required [type=missing, input_value={'EXA_API_KEY': '0f4c29a0...Gt99PY2FK3BTBBGg5vKIyf'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
FIRE_CRAWL_API_KEY
  Field required [type=missing, input_value={'EXA_API_KEY': '0f4c29a0...Gt99PY2FK3BTBBGg5vKIyf'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
TAVLIY_API_KEY
  Field required [type=missing, input_value={'EXA_API_KEY': '0f4c29a0...Gt99PY2FK3BTBBGg5vKIyf'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
LANGFUSE_SECRET_KEY
  Field required [type=missing, input_value={'EXA_API_KEY': '0f4c29a0...Gt99PY2FK3BTBBGg5vKIyf'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
LANGFUSE_PUBLIC_KEY
  Field required [type=missing, input_value={'EXA_API_KEY': '0f4c29a0...Gt99PY2FK3BTBBGg5vK